In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
# Importez les bibliothèques nécessaires
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Chargement des données d'entraînement et de test
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
# Séparez les données en entrées et étiquettes
X = train_data.iloc[:, 1:].values
y = train_data.iloc[:, 0].values


In [ ]:
# Effectuez un split des données d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Prétraitement des données
X_train = X_train.reshape(-1, 180, 180, 1)
X_val = X_val.reshape(-1, 180, 180, 1)
X_test = test_data.values.reshape(-1, 180, 180, 1)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train, num_classes=5270)
y_val = to_categorical(y_val, num_classes=5270)

In [ ]:
# Génération de données supplémentaires
data_generator = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
data_generator.fit(X_train)

In [ ]:
# Créez un modèle de réseau neuronal convolutionnel
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(180,180,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5270, activation='softmax'))

In [ ]:
# Compilez le modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entraînez le modèle
model.fit_generator(data_generator.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train)/32, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# Évaluez le modèle sur les données de validation
val_loss, val_acc = model.evaluate(X_val, y_val)
print("Validation Loss: ", val_loss)
print("Validation Accuracy: ", val_acc)

In [ ]:
# Utilisez le modèle pour prédire les catégories des données de test
test_predictions = model.predict(X_test)

In [ ]:
# Convertir les prédictions en catégories numériques
test_predictions = np.argmax(test_predictions, axis=1)

In [ ]:
# Enregistrer les prédictions dans un fichier CSV pour soumission
submission = pd.DataFrame({'Id': test_data.Id, 'Category': test_predictions})
submission.to_csv("submission.csv", index=False)